In [105]:
n=3

In [106]:
SGA = SymmetricGroupAlgebra(QQ,n)

In [107]:
SGA.dft()

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [87]:
SGA_F7 = SymmetricGroupAlgebra(GF(7),3)

In [88]:
SGA_F7.dft()

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [26]:
SGA_F7.specht_module(Partition([1,1,1]))

Specht module of [(0, 0), (1, 0), (2, 0)] over Finite Field of size 7

In [188]:
SGA_F3 = SymmetricGroupAlgebra(GF(3),3)

In [189]:
#one cannot perform the DFT when p | n!
try:
    SGA_F3.dft()
except ZeroDivisionError:
    print("Modular case not handled!")

Modular case not handled!


In [34]:
#Theorem 3.3, "Nakayama's Conjecture", pg. 8, Wildon, Vertices of Specht Modules and Blocks of the Symmetric Group:

#Let p be a prime. The p-blocks of the symmetric group Sn are labelled by pairs (\gamma,w), 
#where \gamma is a p-core and w \in \mathbb{N}_0 is the associated weight, such that |\gamma| + wp = n. 
#Thus S^\lambda lies in the block labelled by (\gamma,w) if and only if \lambda has p-core \gamma and weight w.
#
#---
#
#Theorem 2.1, Murphy, The Idempotents of the Symmetric Group and Nakayama's Conjecture
#
#F_j^i = \sum_{\mu,k | t_k^\mu \in T_j^i} E_k^\mu 
#H^i = \sum_j F_j^i 
#
#{F_j^i} is a complete set of orthogonal idempotents of RS_n, similarly for {\overline{F}_j^i}.
#
#where
#
#E_i^\mu = \prod_{c = -n+1}^{n-1} \prod_{u | a_{ui}^\mu \ne c} \frac{c-L_u}{c-\alpha_{ui}^\mu}
#
#L_u = (1,u) + (2,u) + ... + (u-1,u), a sum of transpositions
#
#T_j^i are tableaux belonging to equivalence classes B_i, where B_i are equivalences classes of
#partitions given by \tilde_p
#
#\labmda \tidle_p \mu iff the classes (j-i) of each node have the same residue mod p
#
#The class of the node occupied by u in t_i^\mu is \alpha_{ui}^\mu where t_1^\mu, ..., t_d^\mu are the standard \mu-tableaux 
#in the ordering in pg. 288 of "A new construction of Young's seminormal representation of the symmetric groups".
#
#The "class" of a node (i,j) in a Young diagram \mu is the difference j-i.
#
#d is the number of "standard \mu-tableaux", given by the Hook-length formula.
#
#---
#
#Theorem 2.8 (Nakayama's Conjecture): {\overline{H}^i} is a complete set of primitive orthogonal central idempotents of \overline{K}S_n,
#and S_{\overline{K}}^\mu, S_{\overline{K}}^\lambda belong to the same block of \overline{K}S_n if and only if \mu \tilde_p \lambda.
#
#---

In [109]:
mu = Partition([2,1]); mu

[2, 1]

In [113]:
list(StandardTableaux(mu))

[[[1, 3], [2]], [[1, 2], [3]]]

In [61]:
#define the element \alpha_{ui}^\mu
#which is the class of u in the i^th standard Tableaux corresponding to mu
#the class of a node is j-i where i is the row and j is the column
def alpha(mu,u,i):
    tab = StandardTableaux(mu)[i]
    for row in range(len(tab)):
        for col in range(len(tab[row])):
            if u == tab[row][col]:
                return col - row

In [81]:
#define the elment L_u = (1,u) + (2,u) + ... + (u-1,u) in K[S_n]
def L(u):
    return sum(SGA(PermutationGroupElement(f"({i},{u})")) for i in range(1,u))

In [172]:
#create a standard "canonical" tableau by placing entries sequentially from left-to-right, top-to-bottom
def canonical_tableau(la):
    return Tableau([[sum(la[k] for k in range(j))+i+1 for i in range(la[j])] for j in range(len(la))])

In [159]:
def E(mu,i):
    return prod(prod((c-L(u))/(c-alpha(mu,u,i)) for u in range(1,n+1) if alpha(mu,u,i) != c) for c in range(-n+1,n))

In [160]:
E(mu,0)

1/3*[1, 2, 3] + 1/6*[1, 3, 2] - 1/3*[2, 1, 3] - 1/6*[2, 3, 1] - 1/6*[3, 1, 2] + 1/6*[3, 2, 1]

In [173]:
canonical_tableau(mu)

[[1, 2], [3]]

In [190]:
#equivalent to having the same p-core
#look at canonical tableaus (place 1,...,n in top-to-bottom, left-to-right)
#compute p-residues of classes j-i \mod p
def p_equiv(mu_1, mu_2):
    diag_1 = canonical_tableau(mu_1)
    diag_2 = canonical_tableau(mu_2)
    p_residues_1 = [[col - row % p for col in range(len(diag_1[row]))] for row in range(len(diag_1))]
    p_residues_2 = [[col - row % p for col in range(len(diag_2[row]))] for row in range(len(diag_2))]
    #TO-DO

In [192]:
p_equiv(mu,mu,p)

ValueError: [2, 1] is not a 3-core

In [ ]:
def B(n):
    